In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from importlib import reload
import logging
reload(logging)
logging.basicConfig(format='%(asctime)s %(levelname)s:%(message)s', level=logging.INFO, datefmt='%H:%M:%S')

In [3]:
import sys

sys.path.insert(0, '../..')

## Load Dataset

In [4]:
from entity_embed.benchmarks import FodorsZagatsBenchmark

benchmark = FodorsZagatsBenchmark(data_dir_path="../data/")
benchmark

14:52:38 INFO:Extracting Fodors-Zagats...
14:52:38 INFO:Reading Fodors-Zagats row_dict...
14:52:38 INFO:Reading Fodors-Zagats train.csv...
14:52:38 INFO:Reading Fodors-Zagats valid.csv...
14:52:38 INFO:Reading Fodors-Zagats test.csv...


<FodorsZagatsBenchmark> from http://pages.cs.wisc.edu/~anhai/data1/deepmatcher_data/Structured/Fodors-Zagats/fodors_zagat_exp_data.zip

## Preprocess

In [5]:
field_list = ['name', 'addr', 'city', 'phone', 'type']

In [6]:
import unidecode

def clean_str(s):
    return unidecode.unidecode(s).lower().strip()

for record_dict in [benchmark.train_record_dict, benchmark.valid_record_dict, benchmark.test_record_dict]:
    for record in record_dict.values():
        for field in field_list:
            record[field] = clean_str(record[field])

  0%|          | 0/864 [00:00<?, ?it/s]

### Rename attr `type` to `type_` (avoids clash on pytorch)

In [7]:
for row in tqdm(benchmark.row_dict.values()):
    row['type_'] = row['type']
    del row['type']

  0%|          | 0/864 [00:00<?, ?it/s]

## Init Data Module

In [8]:
import torch
import numpy as np

random_seed = 42
torch.manual_seed(random_seed)
np.random.seed(random_seed)

In [11]:
from entity_embed import PairNumericalizer

pair_numericalizer = PairNumericalizer(field_list)

14:52:38 INFO:For attr=name, computing actual max_str_len
14:52:38 INFO:For attr=name, using actual_max_str_len=12
14:52:38 INFO:For attr=addr, computing actual max_str_len
14:52:38 INFO:actual_max_str_len=13 must be even to enable NN pooling. Updating to 14
14:52:38 INFO:For attr=addr, using actual_max_str_len=14
14:52:38 INFO:For attr=city, computing actual max_str_len
14:52:38 INFO:For attr=city, using actual_max_str_len=10
14:52:38 INFO:For attr=phone, computing actual max_str_len
14:52:38 INFO:For attr=phone, using actual_max_str_len=18
14:52:38 INFO:Loading vectors from .vector_cache/wiki.en.vec.pt


{'name': AttrConfig(source_attr='name', field_type=<FieldType.MULTITOKEN: 'multitoken'>, tokenizer='entity_embed.data_utils.numericalizer.default_tokenizer', alphabet=['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', ':', ';', '<', '=', '>', '?', '@', '[', '\\', ']', '^', '_', '`', '{', '|', '}', '~', ' '], max_str_len=12, vocab=None, n_channels=8, embed_dropout_p=0.2, use_attention=True),
 'addr': AttrConfig(source_attr='addr', field_type=<FieldType.MULTITOKEN: 'multitoken'>, tokenizer='entity_embed.data_utils.numericalizer.default_tokenizer', alphabet=['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+'

In [12]:
batch_size = 32
eval_batch_size = 256
datamodule = benchmark.build_matcher_datamodule(
    pair_numericalizer=pair_numericalizer,
    batch_size=batch_size,
    eval_batch_size=eval_batch_size,
    random_seed=random_seed
)

## Training

In [13]:
from entity_embed import Matcher

model = Matcher(
    pair_numericalizer=pair_numericalizer
)

In [14]:
trainer = model.fit(
    datamodule,
    min_epochs=5,
    max_epochs=100,
    check_val_every_n_epoch=1,
    early_stop_monitor="valid_f1_at_0.5",
    tb_save_dir='../tb_logs',
    tb_name=f'matcher-{benchmark.dataset_name}'
)

GPU available: True, used: True
14:52:41 INFO:GPU available: True, used: True
TPU available: None, using: 0 TPU cores
14:52:41 INFO:TPU available: None, using: 0 TPU cores


In [ ]:
model.validate(datamodule)

## Testing

In [17]:
model.test(datamodule)

14:55:04 INFO:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

[{'test_f1_at_0.3': 0.8936169743537903,
  'test_f1_at_0.4': 0.9130434393882751,
  'test_f1_at_0.5': 0.9130434393882751,
  'test_precision_at_0.3': 0.8399999737739563,
  'test_precision_at_0.4': 0.875,
  'test_precision_at_0.5': 0.875,
  'test_recall_at_0.3': 0.9545454382896423,
  'test_recall_at_0.4': 0.9545454382896423,
  'test_recall_at_0.5': 0.9545454382896423}]